In [1]:
import sqlite3
con = sqlite3.connect("tutorial.db")

In [2]:
cur = con.cursor()
cur.execute("CREATE TABLE movie(title, year, score)")
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")
con.commit()
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,)]

In [3]:
%pip install SQLAlchemy --quiet

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import relationship
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, DateTime, String, ForeignKey

engine = create_engine('sqlite:///example', echo=True)

Base = declarative_base()

class Orders(Base):
    __tablename__ = 'orders'

    id = Column(Integer, primary_key=True, nullable=False)
    customer_id = Column(Integer)
    order_date = Column(DateTime)
    status = Column(String)

class OrderDetails(Base):
    __tablename__ = 'order_details'
    
    id = Column(Integer, primary_key=True, nullable=False)
    product_id = Column(Integer)
    quantity = Column(Integer)
    order_id = Column(Integer, ForeignKey(Orders.id), nullable=False)
    order = relationship("Orders")

In [5]:
Base.metadata.create_all(engine)

2024-01-25 17:59:42,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-25 17:59:42,174 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-01-25 17:59:42,175 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-25 17:59:42,177 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2024-01-25 17:59:42,178 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-25 17:59:42,180 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_details")
2024-01-25 17:59:42,180 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-25 17:59:42,181 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("order_details")
2024-01-25 17:59:42,183 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-25 17:59:42,187 INFO sqlalchemy.engine.Engine 
CREATE TABLE orders (
	id INTEGER NOT NULL, 
	customer_id INTEGER, 
	order_date DATETIME, 
	status VARCHAR, 
	PRIMARY KEY (id)
)


2024-01-25 17:59:42,190 INFO sqlalchemy.engine.Engine [no key 0.00326s] ()
2024-01-25 17:59:42,194 INFO sqlalchem

In [6]:
from datetime import datetime
from sqlalchemy.orm import Session

session = Session(engine)
order = Orders(customer_id=192, order_date=datetime.now(), status='P')
session.add(order)
session.commit()

orderDetails1 = OrderDetails(order=order, product_id=4321, quantity=2)
orderDetails2 = OrderDetails(order=order, product_id=5512, quantity=1)
session.add(orderDetails1)
session.add(orderDetails2)
session.commit()

2024-01-25 18:01:04,763 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-25 18:01:04,766 INFO sqlalchemy.engine.Engine INSERT INTO orders (customer_id, order_date, status) VALUES (?, ?, ?)
2024-01-25 18:01:04,766 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (192, '2024-01-25 18:01:04.755460', 'P')
2024-01-25 18:01:04,768 INFO sqlalchemy.engine.Engine COMMIT
2024-01-25 18:01:04,771 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-25 18:01:04,773 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.customer_id AS orders_customer_id, orders.order_date AS orders_order_date, orders.status AS orders_status 
FROM orders 
WHERE orders.id = ?
2024-01-25 18:01:04,773 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (1,)
2024-01-25 18:01:04,775 INFO sqlalchemy.engine.Engine INSERT INTO order_details (product_id, quantity, order_id) VALUES (?, ?, ?) RETURNING id
2024-01-25 18:01:04,776 INFO sqlalchemy.engine.Engine [generated in 0.00008s (insertmanyvalues) 

In [7]:
orderDetails = session.query(OrderDetails).all()
for detail in orderDetails:
    print(detail.quantity)

2024-01-25 18:01:21,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-25 18:01:21,815 INFO sqlalchemy.engine.Engine SELECT order_details.id AS order_details_id, order_details.product_id AS order_details_product_id, order_details.quantity AS order_details_quantity, order_details.order_id AS order_details_order_id 
FROM order_details
2024-01-25 18:01:21,817 INFO sqlalchemy.engine.Engine [generated in 0.00166s] ()
2
1
